In [2]:
import catboost
import xgboost as xgb
import pandas as pd
from sklearn import metrics
import time
import matplotlib.pyplot as plt
import pandas as pd
from os import path
import os
import gc
import numpy as np
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

u_info_header = ["user_id","gender","ocupation","education","marriage","residence"]
l_time_header = ["user_id","grant_time"]
bk_detail_header = ["user_id","tras_time","tras_type","tras_amount","income_sign"]
b_history_header = ["user_id","bh_time","browse_behavior","browse_num"]
bl_detail_header = ["user_id","bill_time","bank_id","last_bill_amount","last_repay_amount","credit_limit",
                    "bill_balance","lowest_repay_limit","consum_num","bill_amount","adjust_amount","cir_interest",
                    "avail_balance","avail_cash_limit","repay_state"]
overdue_header = ["user_id","is_overdue"]
basis_path = "C:\\Users\\CDA\\graduated_design\\data\\train_data\\"
u_info_path = os.path.join(basis_path,"user_info_train.txt")
l_time_path = os.path.join(basis_path,"loan_time_train.txt")
bk_detail_path = os.path.join(basis_path,"bank_detail_train.txt")
bl_detail_path = os.path.join(basis_path,"bill_detail_train.txt")
b_history_path = os.path.join(basis_path,"browse_history_train.txt")
overdue_path = os.path.join(basis_path,"overdue_train.txt")

In [3]:
from catboost import CatBoostClassifier
# Initialize data
cat_features = [0,1,2]
train_data = [["a","b",1,4,5,6],["a","b",4,5,6,7],["c","d",30,40,50,60]]
train_labels = [1,1,-1]
test_data = [["a","b",2,4,6,8],["a","d",1,4,50,60]]
# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=2, learning_rate=1, depth=2, loss_function='Logloss')
# Fit model
model.fit(train_data, train_labels, cat_features)
# Get predicted classes
preds_class = model.predict(test_data)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(test_data)
# Get predicted RawFormulaVal
preds_raw = model.predict(test_data, prediction_type='RawFormulaVal')

0:	learn: 0.2806911	total: 55.4ms	remaining: 55.4ms
1:	learn: 0.1282636	total: 59.9ms	remaining: 0us


In [9]:
#user_info_train = pd.read_table(u_info_path,sep = ",",header = None,names = u_info_header)

overdue_train = pd.read_table(path.join(basis_path,"overdue_train.txt"),sep = ",",header = None, names = ["user_id","is_overdue"])

train_labels = overdue_train["is_overdue"]

train_data = user_info_train#.drop("user_id",axis = 1,inplace = True)
cat_features = np.arange(len(train_data.columns))

In [4]:
final_train = pd.read_csv("C:\\Users\\CDA\\graduated_design\\data\\final_table\\final_total_feature_174.csv")

common_path_1 = "C:\\Users\\CDA\\graduated_design\\data\\train_data\\"

overdue_train = pd.read_table(path.join(common_path_1,"overdue_train.txt"),sep = ",",header = None, names = ["user_id","is_overdue"])

train_labels = overdue_train["is_overdue"]

train_data = final_train.drop("user_id",axis = 1,inplace = True)

cat_features = list(final_train.columns)

In [31]:
train_labels = train_labels.values
train_data = train_data.values
pool = catboost.Pool(train_data,train_labels)
params = {"iterations" :1000,
          "learning_rate" : 0.03,
          "max_depth" : 6, 
          "loss_function" : "Logloss",
          "random_seed" : 7,
          "use_best_model" : True,
          "scale_pos_weight" : 0.2}
scores = catboost.cv(params,pool)

CatboostError: Can't deepcopy _PoolBase object

In [6]:
from sklearn.model_selection import train_test_split
final_train = pd.read_csv("C:\\Users\\CDA\\graduated_design\\data\\final_table\\final_total_feature_174.csv")


common_path_1 = "C:\\Users\\CDA\\graduated_design\\data\\train_data\\"
overdue_train = pd.read_table(os.path.join(common_path_1,"overdue_train.txt"),sep = ",",header = None, names = ["user_id","is_overdue"])
final_train = pd.merge(final_train,overdue_train,how = "inner",on = "user_id")
train_val_xy,offline_xy = train_test_split(final_train,test_size = 0.15,random_state = 7)


train_val_x = train_val_xy.drop(["user_id","is_overdue"],axis = 1)
train_val_y = train_val_xy["is_overdue"]

train_xy,val_xy = train_test_split(train_val_xy,test_size = 0.2,random_state = 7)
train_X = train_xy.drop(["user_id","is_overdue"],axis = 1)
train_y = train_xy["is_overdue"]
val_X = val_xy.drop(["user_id","is_overdue"],axis = 1)
val_y = val_xy["is_overdue"]

In [7]:
cat_model = CatBoostClassifier(iterations =1000,learning_rate = 0.03,depth = 6, loss_function = "Logloss",random_seed = 7,
                           scale_pos_weight = 0.2,l2_leaf_reg = 0.5,rsm = 0.9,)
param_dist_cat = {
    "depth":[5,6],
    "learning_rate":[0.05,0.03]
}
grid_search = GridSearchCV(cat_model,param_grid = param_dist_cat,scoring = "roc_auc",cv = 5,verbose = 10,n_jobs = -1)
grid_search.fit(train_X, train_y)
print(grid_search.best_estimator_)
print(grid_search.score)
print(grid_search.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Program Files\Anaconda3\lib\site-packages\sklearn\base.py:122: DeprecationWarning: Estimator CatBoostClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\base.py:122: DeprecationWarning: Estimator CatBoostClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\base.py:122: DeprecationWarning: Estimator CatBoostClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\base.py:122: DeprecationWarning: Estimator CatBoostClassifier modifies 

0:	learn: 0.6447987	total: 170ms	remaining: 2m 49s
1:	learn: 0.6005418	total: 331ms	remaining: 2m 45s
2:	learn: 0.5600963	total: 517ms	remaining: 2m 51s
3:	learn: 0.5231884	total: 720ms	remaining: 2m 59s
4:	learn: 0.4895340	total: 889ms	remaining: 2m 56s
5:	learn: 0.4588218	total: 1.05s	remaining: 2m 54s
6:	learn: 0.4308410	total: 1.21s	remaining: 2m 51s
7:	learn: 0.4053419	total: 1.39s	remaining: 2m 51s
8:	learn: 0.3821086	total: 1.54s	remaining: 2m 49s
9:	learn: 0.3609047	total: 1.73s	remaining: 2m 50s
10:	learn: 0.3400233	total: 1.9s	remaining: 2m 51s
11:	learn: 0.3222320	total: 2.08s	remaining: 2m 51s
12:	learn: 0.3059375	total: 2.25s	remaining: 2m 50s
13:	learn: 0.2914926	total: 2.4s	remaining: 2m 49s
14:	learn: 0.2783321	total: 2.58s	remaining: 2m 49s
15:	learn: 0.2663370	total: 2.74s	remaining: 2m 48s
16:	learn: 0.2553881	total: 2.9s	remaining: 2m 47s
17:	learn: 0.2453819	total: 3.09s	remaining: 2m 48s
18:	learn: 0.2362071	total: 3.25s	remaining: 2m 47s
19:	learn: 0.2278383	tota

158:	learn: 0.1156372	total: 28s	remaining: 2m 28s
159:	learn: 0.1156160	total: 28.2s	remaining: 2m 28s
160:	learn: 0.1155845	total: 28.4s	remaining: 2m 27s
161:	learn: 0.1155674	total: 28.5s	remaining: 2m 27s
162:	learn: 0.1155489	total: 28.7s	remaining: 2m 27s
163:	learn: 0.1155102	total: 28.9s	remaining: 2m 27s
164:	learn: 0.1154772	total: 29.1s	remaining: 2m 27s
165:	learn: 0.1154315	total: 29.3s	remaining: 2m 27s
166:	learn: 0.1154016	total: 29.5s	remaining: 2m 27s
167:	learn: 0.1153744	total: 29.7s	remaining: 2m 26s
168:	learn: 0.1153639	total: 29.8s	remaining: 2m 26s
169:	learn: 0.1153391	total: 30s	remaining: 2m 26s
170:	learn: 0.1153104	total: 30.2s	remaining: 2m 26s
171:	learn: 0.1152826	total: 30.4s	remaining: 2m 26s
172:	learn: 0.1152658	total: 30.5s	remaining: 2m 26s
173:	learn: 0.1152501	total: 30.7s	remaining: 2m 25s
174:	learn: 0.1152261	total: 30.9s	remaining: 2m 25s
175:	learn: 0.1152095	total: 31s	remaining: 2m 25s
176:	learn: 0.1151741	total: 31.2s	remaining: 2m 25s

316:	learn: 0.1124700	total: 55.3s	remaining: 1m 59s
317:	learn: 0.1124570	total: 55.4s	remaining: 1m 58s
318:	learn: 0.1124513	total: 55.6s	remaining: 1m 58s
319:	learn: 0.1124375	total: 55.8s	remaining: 1m 58s
320:	learn: 0.1124184	total: 55.9s	remaining: 1m 58s
321:	learn: 0.1123990	total: 56.1s	remaining: 1m 58s
322:	learn: 0.1123848	total: 56.3s	remaining: 1m 58s
323:	learn: 0.1123642	total: 56.5s	remaining: 1m 57s
324:	learn: 0.1123414	total: 56.7s	remaining: 1m 57s
325:	learn: 0.1123034	total: 56.8s	remaining: 1m 57s
326:	learn: 0.1122820	total: 57s	remaining: 1m 57s
327:	learn: 0.1122695	total: 57.2s	remaining: 1m 57s
328:	learn: 0.1122393	total: 57.4s	remaining: 1m 56s
329:	learn: 0.1122127	total: 57.5s	remaining: 1m 56s
330:	learn: 0.1121984	total: 57.7s	remaining: 1m 56s
331:	learn: 0.1121876	total: 57.9s	remaining: 1m 56s
332:	learn: 0.1121766	total: 58s	remaining: 1m 56s
333:	learn: 0.1121491	total: 58.2s	remaining: 1m 56s
334:	learn: 0.1121320	total: 58.5s	remaining: 1m 5

471:	learn: 0.1101552	total: 1m 22s	remaining: 1m 31s
472:	learn: 0.1101485	total: 1m 22s	remaining: 1m 31s
473:	learn: 0.1101400	total: 1m 22s	remaining: 1m 31s
474:	learn: 0.1101232	total: 1m 22s	remaining: 1m 31s
475:	learn: 0.1101097	total: 1m 22s	remaining: 1m 31s
476:	learn: 0.1100981	total: 1m 23s	remaining: 1m 31s
477:	learn: 0.1100874	total: 1m 23s	remaining: 1m 30s
478:	learn: 0.1100713	total: 1m 23s	remaining: 1m 30s
479:	learn: 0.1100635	total: 1m 23s	remaining: 1m 30s
480:	learn: 0.1100491	total: 1m 23s	remaining: 1m 30s
481:	learn: 0.1100302	total: 1m 23s	remaining: 1m 30s
482:	learn: 0.1100042	total: 1m 24s	remaining: 1m 30s
483:	learn: 0.1099911	total: 1m 24s	remaining: 1m 29s
484:	learn: 0.1099800	total: 1m 24s	remaining: 1m 29s
485:	learn: 0.1099620	total: 1m 24s	remaining: 1m 29s
486:	learn: 0.1099451	total: 1m 24s	remaining: 1m 29s
487:	learn: 0.1099310	total: 1m 25s	remaining: 1m 29s
488:	learn: 0.1099170	total: 1m 25s	remaining: 1m 28s
489:	learn: 0.1099088	total:

625:	learn: 0.1082126	total: 1m 48s	remaining: 1m 5s
626:	learn: 0.1081982	total: 1m 49s	remaining: 1m 4s
627:	learn: 0.1081886	total: 1m 49s	remaining: 1m 4s
628:	learn: 0.1081780	total: 1m 49s	remaining: 1m 4s
629:	learn: 0.1081696	total: 1m 49s	remaining: 1m 4s
630:	learn: 0.1081626	total: 1m 49s	remaining: 1m 4s
631:	learn: 0.1081514	total: 1m 49s	remaining: 1m 4s
632:	learn: 0.1081347	total: 1m 50s	remaining: 1m 3s
633:	learn: 0.1081230	total: 1m 50s	remaining: 1m 3s
634:	learn: 0.1081123	total: 1m 50s	remaining: 1m 3s
635:	learn: 0.1080953	total: 1m 50s	remaining: 1m 3s
636:	learn: 0.1080836	total: 1m 50s	remaining: 1m 3s
637:	learn: 0.1080709	total: 1m 50s	remaining: 1m 2s
638:	learn: 0.1080594	total: 1m 51s	remaining: 1m 2s
639:	learn: 0.1080502	total: 1m 51s	remaining: 1m 2s
640:	learn: 0.1080381	total: 1m 51s	remaining: 1m 2s
641:	learn: 0.1080260	total: 1m 51s	remaining: 1m 2s
642:	learn: 0.1080161	total: 1m 51s	remaining: 1m 2s
643:	learn: 0.1079954	total: 1m 52s	remaining:

782:	learn: 0.1064737	total: 2m 15s	remaining: 37.6s
783:	learn: 0.1064638	total: 2m 16s	remaining: 37.5s
784:	learn: 0.1064579	total: 2m 16s	remaining: 37.3s
785:	learn: 0.1064527	total: 2m 16s	remaining: 37.1s
786:	learn: 0.1064311	total: 2m 16s	remaining: 37s
787:	learn: 0.1064209	total: 2m 16s	remaining: 36.8s
788:	learn: 0.1064067	total: 2m 16s	remaining: 36.6s
789:	learn: 0.1063951	total: 2m 17s	remaining: 36.4s
790:	learn: 0.1063821	total: 2m 17s	remaining: 36.3s
791:	learn: 0.1063629	total: 2m 17s	remaining: 36.1s
792:	learn: 0.1063577	total: 2m 17s	remaining: 35.9s
793:	learn: 0.1063438	total: 2m 17s	remaining: 35.7s
794:	learn: 0.1063314	total: 2m 17s	remaining: 35.6s
795:	learn: 0.1063231	total: 2m 18s	remaining: 35.4s
796:	learn: 0.1063160	total: 2m 18s	remaining: 35.2s
797:	learn: 0.1063079	total: 2m 18s	remaining: 35s
798:	learn: 0.1063017	total: 2m 18s	remaining: 34.9s
799:	learn: 0.1062881	total: 2m 18s	remaining: 34.7s
800:	learn: 0.1062833	total: 2m 18s	remaining: 34.

939:	learn: 0.1048294	total: 2m 43s	remaining: 10.4s
940:	learn: 0.1048169	total: 2m 43s	remaining: 10.2s
941:	learn: 0.1047991	total: 2m 43s	remaining: 10.1s
942:	learn: 0.1047851	total: 2m 43s	remaining: 9.89s
943:	learn: 0.1047687	total: 2m 43s	remaining: 9.71s
944:	learn: 0.1047575	total: 2m 43s	remaining: 9.54s
945:	learn: 0.1047456	total: 2m 44s	remaining: 9.37s
946:	learn: 0.1047311	total: 2m 44s	remaining: 9.2s
947:	learn: 0.1047187	total: 2m 44s	remaining: 9.02s
948:	learn: 0.1047088	total: 2m 44s	remaining: 8.85s
949:	learn: 0.1046979	total: 2m 44s	remaining: 8.68s
950:	learn: 0.1046909	total: 2m 45s	remaining: 8.5s
951:	learn: 0.1046755	total: 2m 45s	remaining: 8.33s
952:	learn: 0.1046664	total: 2m 45s	remaining: 8.16s
953:	learn: 0.1046637	total: 2m 45s	remaining: 7.98s
954:	learn: 0.1046586	total: 2m 45s	remaining: 7.81s
955:	learn: 0.1046515	total: 2m 45s	remaining: 7.64s
956:	learn: 0.1046404	total: 2m 46s	remaining: 7.46s
957:	learn: 0.1046325	total: 2m 46s	remaining: 7

In [23]:
for i in list(final_train.columns):
    print(model.get_object_importance(i))
#print(sorted(model.feature_importances_))
print(model.tree_count_)

TypeError: get_object_importance() missing 1 required positional argument: 'train_pool'

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172])